In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [4]:
batch_size = 64 # Batch size for training
epochs = 100 # Number of epochs for training
latent_dim =  256 # Latent dimensionality of the encoding space
num_samples = 10000 # Number of training samples

In [5]:
data_path = '/content/drive/MyDrive/fra.txt'

In [6]:
# Vectorize the data

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding = 'utf-8') as f:
  lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines)-1)]:
  input_text, target_text, _ = line.split('\t')
  # We use 'tab' as the 'start sequence' character for the targets and '\n' as 'end sequence character'

  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)

  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [7]:
input_characters

{' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é'}

In [8]:
input_texts

['Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Who?',
 'Wow!',
 'Fire!',
 'Help!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Attack!',
 'Attack!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Get up.',
 'Go now.',
 'Go now.',
 'Go now.',
 'Got it!',
 'Got it!',
 'Got it?',
 'Got it?',
 'Got it?',
 'Hop in.',
 'Hop in.',
 'Hug me.',
 'Hug me.',
 'I fell.',
 'I fell.',
 'I know.',
 'I left.',
 'I left.',
 'I lied.',
 'I lost.',
 'I paid.',
 "I'm 19.",
 "I'm OK.",
 "I'm OK.",
 'Listen.',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'Really?',
 'Really?',
 'Really?',
 'Thanks.',
 'We try.',
 'We won.',
 'We won.',
 'We won.',
 'We won.',
 'Ask Tom.',
 'Awesome!',
 'Be calm.',
 'Be calm.',
 'Be calm.',
 'Be cool.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fai

In [9]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '’',
 '\u202f'}

In [10]:
len(target_characters)

92

In [11]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [12]:
print('Number of samples: ', len(input_texts))
print('Number of unique tokens: ', num_encoder_tokens)
print('Number of uniqie output tokens: ', num_decoder_tokens)
print('Max sequence length for inputs: ', max_encoder_seq_length)
print('Max sequence length for outputs: ', max_decoder_seq_length)

Number of samples:  10000
Number of unique tokens:  71
Number of uniqie output tokens:  92
Max sequence length for inputs:  16
Max sequence length for outputs:  59


In [13]:
input_token_index = dict(
    [(char,i) for i, char in enumerate(input_characters)])

target_token_index = dict(
    [(char,i) for i, char in enumerate(target_characters)])

In [14]:
input_token_index, target_token_index

({' ': 0,
  '!': 1,
  '$': 2,
  '%': 3,
  '&': 4,
  "'": 5,
  ',': 6,
  '-': 7,
  '.': 8,
  '0': 9,
  '1': 10,
  '2': 11,
  '3': 12,
  '5': 13,
  '6': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  'é': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  '(': 7,
  ')': 8,
  ',': 9,
  '-': 10,
  '.': 11,
  '0': 12,
  '1': 13,
  '2': 14,
  '3': 15,
  '5': 16,
  '8': 17,
  '9': 18,
  ':': 19,
  '?': 20,
  'A'

In [15]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')


*Implementing onehot representation*

In [16]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1
  encoder_input_data[i, t+1:, input_token_index[' ']] = 1

  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1
    if t>0:
      #decoder_target_data will be ahead by one timestep and will not include the start character
      decoder_target_data[i, t-1, target_token_index[char]] = 1
  
  decoder_input_data[i, t+1:, target_token_index[' ']] = 1
  decoder_target_data[i, t:, target_token_index[' ']] = 1

In [17]:
encoder_input_data[0].shape

(16, 71)

In [18]:
# Define an input sequence and process it
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard 'encoder_outputs' and only keep the states

encoder_states = [state_h, state_c]

In [19]:
# Set up the decoder using 'encoder_states' as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences, and to return internal states as well
# We don't use the return states in the training model, but we will use them in inference

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [20]:
# Set up the model that will turn 'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training 
model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size,
          epochs = epochs, validation_split = 0.2)

Epoch 1/100
125/125 [==============================] - 13s 46ms/step - loss: 1.1671 - accuracy: 0.7252 - val_loss: 1.0978 - val_accuracy: 0.6996
Epoch 2/100
125/125 [==============================] - 4s 35ms/step - loss: 0.8480 - accuracy: 0.7708 - val_loss: 0.8382 - val_accuracy: 0.7670
Epoch 3/100
125/125 [==============================] - 4s 35ms/step - loss: 0.6742 - accuracy: 0.8084 - val_loss: 0.7132 - val_accuracy: 0.7885
Epoch 4/100
125/125 [==============================] - 4s 35ms/step - loss: 0.5906 - accuracy: 0.8285 - val_loss: 0.6484 - val_accuracy: 0.8108
Epoch 5/100
125/125 [==============================] - 4s 35ms/step - loss: 0.5418 - accuracy: 0.8420 - val_loss: 0.6067 - val_accuracy: 0.8221
Epoch 6/100
125/125 [==============================] - 4s 34ms/step - loss: 0.5052 - accuracy: 0.8524 - val_loss: 0.5761 - val_accuracy: 0.8298
Epoch 7/100
125/125 [==============================] - 4s 35ms/step - loss: 0.4766 - accuracy: 0.8599 - val_loss: 0.5466 - val_accuracy

*Accuracy of Encoder and Decoder model on validation set which makes use of LSTM is 87.44%*

Next: Inference mode (Sampling)

1. Encode input and retrieve initial decoder state

2. Run one step of decoder with this initial state and a 'start of sequence' token as target. Output will be the next target token

3. Repeat with the current target token and current states 

In [24]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
     [decoder_outputs] + decoder_states)



Reverse - Lookup token index to decode sequences back to something readable

In [25]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())

reverse_target_char_index = dict(
    (i, char) for char,i in target_token_index.items())

In [30]:
def decode_sequence(input_seq):
  # Encode the input as state vectors
  states_value = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1
  target_seq = np.zeros((1,1,num_decoder_tokens))

  # Populate the first character of target sequence with the start character
  target_seq[0,0,target_token_index['\t']] = 1

  # Sampling loop for a batch of sequences (to simplify, here we assume batch of size 1)
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
        [target_seq] + states_value)
    
    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0,-1,:])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char

    # Exit condition: either hit max length or find stop character
    if(sampled_char == '\n' or 
       len(decoded_sentence) > max_decoder_seq_length):
        stop_condition = True

    
    # Update the target sequence (of length 1)
    target_seq = np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,sampled_token_index] = 1

    # Update states
    states_value = [h,c]

  return decoded_sentence




In [31]:
for seq_index in range(100):
  # Take one sequence (part of the training set) for trying out decoding

  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Hello!
Decoded sentence: Salut !

-